In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [110]:
import os
import pandas as pd
from gensim.models import Doc2Vec
workdir_path = '/content/drive/My Drive/' + '00 PUC BI MASTER/00 - PROJ (TCC)/PREDIÇÃO ATIVIDADES/event2mind'  # Inserir o local da pasta onde estão os arquivos de entrada (treino e teste)
os.chdir(workdir_path)

MODEL LOAD - INPUT

In [111]:
#Parametros de Input
model = Doc2Vec.load("Event2Mind_Events.model")
input_description = pd.read_csv('embeddingsEvents.csv', sep = ';')

campo_input = "Event"

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


OUTPUT - CLUSTER


In [172]:
#Parametros Output
cluster_output = pd.read_csv('clusterIntencoes.csv', sep = ',')
cluster_output.rename(columns={"Xintent": "Xintent_2"}, inplace = True)
cluster_output.rename(columns={"cluster_intents": "cluster_output"}, inplace = True)

campo_output = "Xintent_2"

In [173]:
print(cluster_output['cluster_output'].unique().tolist())

[5, 3, 6, 1, 0, 2, 4]


TRATAMENTOS BASE

In [117]:
input_description['input_index'] = list(range(len(input_description.index)))
input_description['input_index'] = input_description['input_index'] + 1
embeddings_input = model.docvecs.vectors_docs

def removeCaracteres(nomeColuna, dataSet):
  dataSet[nomeColuna]= dataSet[nomeColuna].astype(str)
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("`` ve", "")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("`` s", "")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("'", "")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("&", " ")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace(",", " ")
  dataSet[nomeColuna] = dataSet[nomeColuna].str.replace("  ", " ")


CRUZANDO CLUSTER DE INTEÇÕES COM EVENTOS PARA DEIXA-LOS NA MESMA DIMENSÃO

In [174]:
#BASE COM TODAS AS INTENÇÕES E EMOÇÕES
dtPrincipal = pd.read_csv('event2MindClean.csv', sep = ',')
dtPrincipal = dtPrincipal[dtPrincipal[campo_output] != "none"]
dtPrincipal = dtPrincipal[dtPrincipal[campo_output] != "'none'"]
removeCaracteres(campo_output, dtPrincipal)

#set_index = input variable
dtPrincipal = dtPrincipal.join(input_description.set_index(campo_input), on = campo_input, how = 'inner')

#set_index = output variable
dtPrincipal = cluster_output.join(dtPrincipal.set_index(campo_output), on = campo_output, how = 'inner')  

#criando novo index
dtPrincipal['new_index'] = list(range(len(dtPrincipal.index)))
dtPrincipal['new_index'] = dtPrincipal['new_index'] + 1
dtPrincipal.set_index(['new_index'], inplace = True)

len(dtPrincipal)

92405

In [175]:
dtPrincipal.head(5)

,cluster_output,Xintent_2,Indice,Xintent,Xemotion,Event,Xsent,Osent,Event_2,Xemotion_2,vetor_doc_str,cluster_events,input_index
new_index,,,,,,,,,,,,,
1,5,advantage,773,to have an advantage,pleased,PersonX uses PersonX's ___ to obtain,3.0,2.0,"'personx, ', ',', 'us, ', ',', 'personx, ', ',...",'pleased ',[-0.47499427 0.0606029 0.01173875 0.530232...,17,329
2,5,advantage,773,to have an advantage,smug,PersonX uses PersonX's ___ to obtain,3.0,2.0,"'personx, ', ',', 'us, ', ',', 'personx, ', ',...",'smug ',[-0.47499427 0.0606029 0.01173875 0.530232...,17,329
3,5,advantage,773,to have an advantage,excited,PersonX uses PersonX's ___ to obtain,3.0,2.0,"'personx, ', ',', 'us, ', ',', 'personx, ', ',...",'excited ',[-0.47499427 0.0606029 0.01173875 0.530232...,17,329
4,5,advantage,8698,advantage,good,PersonX takes ___ of opportunities,4.0,5.0,"'personx, ', ',', 'tak, ', ',', ', ', ',', 'op...",'good ',[-3.53230327e-01 1.98501185e-01 -6.00352824e-...,1,4208
5,5,advantage,16524,to have an advantage,in control,PersonX throws off balance,2.0,2.0,"'personx, ', ',', 'throw, ', ',', 'balanc, '",'control ',[ 0.26132074 0.02934684 0.38985577 -0.279397...,16,7719


CRIANDO NUMPY ARRAYS COM OS INPUTS E OUTPUTS

In [176]:
#criar um embeddings final que segue a ordem do input no dtPrincipal
embeddingDupl = []
cluster = []

for i in range(1, len(dtPrincipal)):

  linhaEmbedding = (dtPrincipal['input_index'][i]) - 1
  embeddingDupl.append(embeddings_input[linhaEmbedding])
  cluster.append(dtPrincipal['cluster_output'][i])

len(embeddingDupl)

92404

RODAR ABAIXO CASO QUEIRA CRIAR UMA CLASSE COM BASE NAS PALAVRAS

In [ ]:
'''
listPalavras = ['happy', 'glad', 'excited', 'relieved', 'show', 'know', 'go', 'give', 'take', 'keep', 'fell', 'enjoy', 'work', 'wants', 'find', 'look', 'avoid']
listOutput = dtPrincipal[campo_output].toList()

import math

classes = []

for i in range(0, len(listOutput):
  texto = listOutput[i]
  classeTemp = []
  for z in range(0, len(listPalavras)):
       if (texto.find(listPalavras[z]) != -1):
        flag = z
        classeTemp.append(flag)
         
    
  classeFinal = pd.Series(classeTemp).min()
  classes.append(classeFinal)

maximo = pd.Series(classes).max()    
classes = [ maximo + 1 if math.isnan(x) else x for x in classes] 

cluster_output = cluster_output.assign(classes=classes)

SEPARANDO INPUT (X) E OUTPUT (Y) COMO NUMPY ARRAY

In [177]:
import numpy as np

X = np.array(embeddingDupl)
Y = np.array(cluster)
X.shape

(92404, 400)

IMPORTS

In [178]:
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.model_selection import train_test_split

SPLIT DA BASE EM TREINO E TESTE

In [179]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

NORMALIZAÇÃO E SCALER

In [180]:
scaler = StandardScaler()
scaler_model = scaler.fit(x_train)

x_train_scaled = scaler_model.transform(x_train)
x_test_scaled = scaler_model.transform(x_test)

pcaComp = PCA(n_components = 0.99)
pcaModel = pcaComp.fit(x_train_scaled)
x_train_prepared = pcaModel.transform(x_train_scaled)
x_test_prepared = pcaModel.transform(x_test_scaled)

#caso não opte por testar com normalização e Scaler
#x_train_prepared = x_train
#x_test_prepared = x_test


OBTENDO AS DIMENSÕES

In [181]:
dimensao_x = x_train_prepared.shape[1]
camadas_saida  = len(dtPrincipal['cluster_output'].unique())

print("dimensao x: %(dx)s e dimensao y: %(dy)s"% {'dx': dimensao_x, 'dy': camadas_saida} )

dimensao x: 84 e dimensao y: 7


PARAMETROS MODELOS

In [182]:
epocas = 500

#nomes dos modelos
modelo1 = 'MLP 1 Camada'
modelo2 = 'MLP 2 Camadas'
modelo3 = 'KNN 2'
modelo4 = 'LSTM 1 Camada'
modelo5 = 'LSTM 2 Camadas'
modelo6 = 'LSTM 3 Camadas'

DEFININDO CAMADAS DOS MODELOS

In [183]:
for k in range(3, 10):
  potencia = 2 ** k
  if potencia < dimensao_x:
    primeiro_neuronio_lstm = potencia

for k in range(3, 10):
  potencia = 2 ** k
  if potencia < primeiro_neuronio_lstm:
    segundo_neuronio_lstm = potencia

for k in range(3, 10):
  potencia = 2 ** k
  if potencia < segundo_neuronio_lstm:
    terceiro_neuronio_lstm = potencia

print("O primeiro neuronio do modelo de LSTM tera %(n1)s camadas. O segundo neuronio terá %(n2)s camadas. O terceiro neuronio terá %(n3)s camadas." % {'n1': primeiro_neuronio_lstm, 'n2': segundo_neuronio_lstm, 'n3': terceiro_neuronio_lstm})

O primeiro neuronio do modelo de LSTM tera 64 camadas. O segundo neuronio terá 32 camadas. O terceiro neuronio terá 16 camadas.


In [184]:
primeira_hl_mlp = round((dimensao_x + camadas_saida) / 2)
segunda_hl_mlp = round(primeira_hl_mlp /2)

print("O primeiro neuronio do modelo de MLP tera %(n1)s camadas. O segundo neuronio terá %(n2)s camadas" % {'n1': primeira_hl_mlp, 'n2': segunda_hl_mlp})

O primeiro neuronio do modelo de MLP tera 46 camadas. O segundo neuronio terá 23 camadas




MULTILAYER PERCEPTRON 1 CAMADA


In [185]:
clf_mlp1 = MLPClassifier(solver='sgd'
                     , hidden_layer_sizes=(primeira_hl_mlp)
                     , max_iter = (epocas * 2)
                     , random_state=1)

clf_mlp1.out_activation_ = 'softmax'

clf_mlp1.fit(x_train_prepared, y_train)
y_predicted_mlp1 = clf_mlp1.predict(x_test_prepared)

accuracy_modelo1 = metrics.accuracy_score(y_test, y_predicted_mlp1).round(3)
print(accuracy_modelo1)

0.237


In [186]:
y_predicted_mlp1.max()

6

In [187]:
mlp1_results = pd.DataFrame(list(zip(y_predicted_mlp1, y_test)), columns =['predito', 'real'])
df_confusion = pd.crosstab(mlp1_results.real, mlp1_results.predito)

export_path = workdir_path + '/matrizConfusaoMLP1.csv'
df_confusion.to_csv (export_path, index = True, header=True)

In [188]:
df_confusion.head(20)

predito,0,1,2,3,4,5,6
real,,,,,,,
0,891,54,979,23,762,179,152
1,384,77,694,12,449,112,86
2,673,66,1814,21,781,158,156
3,335,26,636,28,397,74,79
4,728,58,1183,22,1149,215,184
5,543,34,898,22,690,245,131
6,476,26,918,15,539,130,177


MULTILAYER PERCEPTRON 2 CAMADAS

In [189]:
clf_mlp2 = MLPClassifier(solver='sgd'
                     , hidden_layer_sizes=(primeira_hl_mlp, segunda_hl_mlp)
                     , max_iter = (epocas * 2)
                     , random_state=1)

clf_mlp2.out_activation_ = 'softmax'

clf_mlp2.fit(x_train_prepared, y_train)
y_predicted_mlp2 = clf_mlp2.predict(x_test_prepared)

accuracy_modelo2 = metrics.accuracy_score(y_test, y_predicted_mlp2).round(3)
print(accuracy_modelo2)

0.237


NEAREST NEIGHBOR


In [190]:
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(x_train_prepared, y_train)
y_predicted_neigh = neigh.predict(x_test_prepared)

accuracy_modelo3 = metrics.accuracy_score(y_test, y_predicted_neigh).round(3)
print(accuracy_modelo3)

0.313


RESHAPING PARA USO DOS MODELOS DOS KERAS

In [191]:
#Reshaping
X_train = np.reshape(x_train_prepared, (x_train_prepared.shape[0], 1, x_train_prepared.shape[1]))
X_test = np.reshape(x_test_prepared, (x_test_prepared.shape[0], 1, x_test_prepared.shape[1]))

MODELO DE REDE NEURAL LSTM - 1 CAMADA ESCONDIDA

In [192]:
#Initicializar a RNN
regressor = Sequential()
 
# Adicionar a primeira camada LSTM e Dropout 
regressor.add(LSTM(units = primeiro_neuronio_lstm, input_shape=(1, x_train_prepared.shape[1])))
regressor.add(Dropout(0.2))
 
# camada de saída
regressor.add(Dense(units = camadas_saida, activation='softmax')) #para classificação dar como entrada as classes com função de ativação softmax

# Compilar a rede
regressor.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Visualizar a rede
regressor.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 64)                38144     
_________________________________________________________________
dropout_9 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 455       
Total params: 38,599
Trainable params: 38,599
Non-trainable params: 0
_________________________________________________________________


In [193]:
regressor.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = epocas, batch_size = 32)

Train on 73923 samples, validate on 18481 samples
Epoch 1/500
73923/73923 [==============================] - 5s 74us/step - loss: 1.8998 - accuracy: 0.2119 - val_loss: 1.8828 - val_accuracy: 0.2253
Epoch 2/500
73923/73923 [==============================] - 5s 70us/step - loss: 1.8646 - accuracy: 0.2393 - val_loss: 1.8718 - val_accuracy: 0.2371
Epoch 3/500
73923/73923 [==============================] - 5s 70us/step - loss: 1.8410 - accuracy: 0.2562 - val_loss: 1.8625 - val_accuracy: 0.2427
Epoch 4/500
73923/73923 [==============================] - 5s 68us/step - loss: 1.8190 - accuracy: 0.2715 - val_loss: 1.8546 - val_accuracy: 0.2489
Epoch 5/500
73923/73923 [==============================] - 5s 70us/step - loss: 1.8015 - accuracy: 0.2824 - val_loss: 1.8514 - val_accuracy: 0.2525
Epoch 6/500
73923/73923 [==============================] - 5s 68us/step - loss: 1.7867 - accuracy: 0.2898 - val_loss: 1.8494 - val_accuracy: 0.2566
Epoch 7/500
73923/73923 [==============================] - 5s 

In [194]:
acur_test = regressor.evaluate(X_test, y_test, verbose=0)
accuracy_modelo4 = acur_test[1]
print(accuracy_modelo4)

0.29960501194000244


MATRIZ DE CONFUSÃO

In [195]:
y_predicted_lstm = regressor.predict_classes(X_test)
lstm_results = pd.DataFrame(list(zip(y_predicted_lstm, y_test)), columns =['predito', 'real'])
df_confusion = pd.crosstab(lstm_results.real, lstm_results.predito)

export_path = workdir_path + '/matrizConfusaoLSTM1.csv'
df_confusion.to_csv (export_path, index = True, header=True)

MODELO DE REDES NEURAL LSTM - 2 CAMADAS ESCONDIDAS

In [196]:
# Initicializar a RNN
regressor = Sequential()
 
# Adicionar a primeira camada LSTM e Dropout 
regressor.add(LSTM(units = primeiro_neuronio_lstm, return_sequences = True, input_shape=(1, x_train_prepared.shape[1])))
regressor.add(Dropout(0.2))
 
# Adicionar a terceira camada LSTM e Dropout
regressor.add(LSTM(units = segundo_neuronio_lstm))
regressor.add(Dropout(0.2))
 
# camada de saída
regressor.add(Dense(units = camadas_saida, activation='softmax')) #para classificação dar como entrada as classes com função de ativação softmax

# Compilar a rede
regressor.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Visualizar a rede
regressor.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 1, 64)             38144     
_________________________________________________________________
dropout_10 (Dropout)         (None, 1, 64)             0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dropout_11 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 7)                 231       
Total params: 50,791
Trainable params: 50,791
Non-trainable params: 0
_________________________________________________________________


In [197]:
#fitting
regressor.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = epocas, batch_size = 32 )

Train on 73923 samples, validate on 18481 samples
Epoch 1/500
73923/73923 [==============================] - 8s 105us/step - loss: 1.8965 - accuracy: 0.2101 - val_loss: 1.8884 - val_accuracy: 0.2224
Epoch 2/500
73923/73923 [==============================] - 7s 94us/step - loss: 1.8714 - accuracy: 0.2330 - val_loss: 1.8744 - val_accuracy: 0.2299
Epoch 3/500
73923/73923 [==============================] - 7s 94us/step - loss: 1.8466 - accuracy: 0.2519 - val_loss: 1.8620 - val_accuracy: 0.2415
Epoch 4/500
73923/73923 [==============================] - 7s 94us/step - loss: 1.8245 - accuracy: 0.2655 - val_loss: 1.8552 - val_accuracy: 0.2464
Epoch 5/500
73923/73923 [==============================] - 7s 96us/step - loss: 1.8062 - accuracy: 0.2793 - val_loss: 1.8485 - val_accuracy: 0.2517
Epoch 6/500
73923/73923 [==============================] - 7s 97us/step - loss: 1.7921 - accuracy: 0.2851 - val_loss: 1.8440 - val_accuracy: 0.2562
Epoch 7/500
73923/73923 [==============================] - 7s

In [198]:
acur_test = regressor.evaluate(X_test, y_test, verbose=0)
accuracy_modelo5 = acur_test[1]
print(accuracy_modelo5)

0.30068719387054443


MATRIZ DE CONFUSÃO

In [199]:
y_predicted_lstm = regressor.predict_classes(X_test)
lstm_results = pd.DataFrame(list(zip(y_predicted_lstm, y_test)), columns =['predito', 'real'])
df_confusion = pd.crosstab(lstm_results.real, lstm_results.predito)

export_path = workdir_path + '/matrizConfusaoLSTM2.csv'
df_confusion.to_csv (export_path, index = True, header=True)

CRIANDO A REDE COM 3 CAMADAS

In [200]:
# Initicializar a RNN
regressor = Sequential()
 
# Adicionar a primeira camada LSTM e Dropout 
regressor.add(LSTM(units = primeiro_neuronio_lstm, return_sequences = True, input_shape=(1, x_train_prepared.shape[1])))
regressor.add(Dropout(0.2))
 
# Adicionar a segunda camada LSTM e Dropout
regressor.add(LSTM(units = segundo_neuronio_lstm, return_sequences = True))
regressor.add(Dropout(0.2))
 
# Adicionar a terceira camada LSTM e Dropout
regressor.add(LSTM(units = terceiro_neuronio_lstm))
regressor.add(Dropout(0.2))
 
# camada de saída
regressor.add(Dense(units = camadas_saida, activation='softmax')) #para classificação dar como entrada as classes com função de ativação softmax

# Compilar a rede
regressor.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Visualizar a rede
regressor.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 1, 64)             38144     
_________________________________________________________________
dropout_12 (Dropout)         (None, 1, 64)             0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 1, 32)             12416     
_________________________________________________________________
dropout_13 (Dropout)         (None, 1, 32)             0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 16)                3136      
_________________________________________________________________
dropout_14 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 7)                

In [201]:
#fitting
regressor.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = epocas, batch_size = 32 )

Train on 73923 samples, validate on 18481 samples
Epoch 1/500
73923/73923 [==============================] - 11s 146us/step - loss: 1.8990 - accuracy: 0.2087 - val_loss: 1.8939 - val_accuracy: 0.2154
Epoch 2/500
73923/73923 [==============================] - 10s 132us/step - loss: 1.8801 - accuracy: 0.2297 - val_loss: 1.8802 - val_accuracy: 0.2300
Epoch 3/500
73923/73923 [==============================] - 9s 124us/step - loss: 1.8599 - accuracy: 0.2464 - val_loss: 1.8711 - val_accuracy: 0.2367
Epoch 4/500
73923/73923 [==============================] - 9s 126us/step - loss: 1.8423 - accuracy: 0.2576 - val_loss: 1.8643 - val_accuracy: 0.2421
Epoch 5/500
73923/73923 [==============================] - 9s 123us/step - loss: 1.8274 - accuracy: 0.2704 - val_loss: 1.8601 - val_accuracy: 0.2491
Epoch 6/500
73923/73923 [==============================] - 9s 120us/step - loss: 1.8169 - accuracy: 0.2758 - val_loss: 1.8548 - val_accuracy: 0.2501
Epoch 7/500
73923/73923 [=============================

In [202]:
acur_test = regressor.evaluate(X_test, y_test, verbose=0)
accuracy_modelo6 = acur_test[1]
print(accuracy_modelo6)

0.30122828483581543


MATRIZ DE CONFUSÃO

In [203]:
y_predicted_lstm = regressor.predict_classes(X_test)
lstm_results = pd.DataFrame(list(zip(y_predicted_lstm, y_test)), columns =['predito', 'real'])
df_confusion = pd.crosstab(lstm_results.real, lstm_results.predito)

export_path = workdir_path + '/matrizConfusaoLSTM3.csv'
df_confusion.to_csv (export_path, index = True, header=True)

EXPORTANDO RESULTADOS DE ACURACIA

In [204]:
descricoes_modelos = [modelo1, modelo2, modelo3, modelo4, modelo5, modelo6]
acuracias = [accuracy_modelo1, accuracy_modelo2, accuracy_modelo3, accuracy_modelo4, accuracy_modelo5, accuracy_modelo6]


In [205]:

acuracias_df = pd.DataFrame(list(zip(descricoes_modelos, acuracias)), 
               columns =['modelo', 'acuracia']) 

export_path = workdir_path + '/acuracias.csv'
acuracias_df.to_csv (export_path, index = True, header=True)